In [1]:
from vllm.v1.worker.gpu_model_runner import GPUModelRunner
GPUModelRunner._prepare_inputs??

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 12-07 16:12:25 [importing.py:44] Triton is installed but 0 active driver(s) found (expected 1). Disabling Triton to prevent runtime errors.
INFO 12-07 16:12:25 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 12-07 16:12:25 [interface.py:221] Failed to import from vllm._C: ImportError('libcuda.so.1: cannot open shared object file: No such file or directory')


Signature:
GPUModelRunner._prepare_inputs(
    self,
    scheduler_output: 'SchedulerOutput',
    num_scheduled_tokens: numpy.ndarray,
) -> tuple[torch.Tensor, vllm.v1.spec_decode.metadata.SpecDecodeMetadata | None]
Source:   
    def _prepare_inputs(
        self,
        scheduler_output: "SchedulerOutput",
        num_scheduled_tokens: np.ndarray,
    ) -> tuple[
        torch.Tensor,
        SpecDecodeMetadata | None,
    ]:
        """
        :return: tuple[
            logits_indices, spec_decode_metadata,
        ]
        """
        total_num_scheduled_tokens = scheduler_output.total_num_scheduled_tokens
        assert total_num_scheduled_tokens > 0
        num_reqs = self.input_batch.num_reqs
        assert num_reqs > 0

        # OPTIMIZATION: Start copying the block table first.
        # This way, we can overlap the copy with the following CPU operations.
        self.input_batch.block_table.commit_block_table(num_reqs)

        # Get request indices.
        # E.g., [2, 